# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [23]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pandas as pd
import json
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
city_data = "../WeatherPy/city_weather.csv"
data_file = pd.read_csv(city_data)
data_file

City    Lat     Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
0     Hermanus -34.42   19.23     61.00        87          99        3.00   
1    Busselton -33.65  115.33     75.00        46           0       12.46   
2        Bluff -46.60  168.33     54.00        59          73        8.01   
3        Vaini -21.20 -175.20     73.40        94          90       13.87   
4        Kapaa  22.08 -159.32     82.99        74          40        9.17   
..         ...    ...     ...       ...       ...         ...         ...   
573     Surgut  61.25   73.42     26.60       100          90       11.18   
574      Mugan  35.94  101.38     39.61        65          87        6.06   
575    Kurchum  48.57   83.65     35.38        72          97        9.10   
576  Maniitsoq  65.42  -52.90     28.13        79         100        8.28   
577    Tongren  27.72  109.19     59.74        86         100        0.72   

    Country        Date  
0        ZA  1603672968  
1        AU  1603672968  
2        NZ  1603672494  
3        TO  1603672471  
4        US  1603672489  
..      ...         ...  
573      RU  1603673059  
574      CN  1603673059  
575      KZ  1603673059  
576      GL  1603673059  
577      CN  1603673059  

[578 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [31]:
gmaps.configure(api_key=g_key)

locations = data_file[['Lat', 'Lng']]
weights = data_file['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
temp_first = data_file.loc[(data_file["Max Temp"] >= 68) & (data_file["Max Temp"] < 75), :]
wind_second = temp_first.loc[temp_first["Wind Speed"] < 10, :]
humidity_third = wind_second.loc[(wind_second["Humidity"] >= 30) & (wind_second["Humidity"] < 70), :]
cloudy_fourth = humidity_third.loc[humidity_third["Cloudiness"] < 40 ,:]
cloudy_fourth.count()

# dropna not used as .count shows that all rows have the same amount of values 



City          11
Lat           11
Lng           11
Max Temp      11
Humidity      11
Cloudiness    11
Wind Speed    11
Country       11
Date          11
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = cloudy_fourth

hotel_df["Hotel Name"] = ""
# hotel_df["Hotel Lat"] = ""
# hotel_df["Hotel Lng"] = ""
hotel_df



C:\Users\MSI\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
18     Ponta do Sol  32.67  -17.10     68.00        64          20   
24   Saint-Philippe -21.36   55.77     71.60        68           0   
176    Jagannāthpur  22.23   85.65     68.22        60           0   
335    Saint-Pierre -21.34   55.48     69.80        68           0   
347            Esna  25.29   32.55     69.80        52           0   
453           Simao  22.79  100.97     69.03        66          20   
460            Tezu  27.92   96.17     74.32        66           0   
481         Sheopur  25.67   76.70     68.32        40           0   
494    Saint-Joseph -21.37   55.62     69.80        68           0   
514            Bara  10.37   10.73     73.78        36          30   
557      Thabazimbi -24.59   27.41     70.00        66           0   

     Wind Speed Country        Date Hotel Name  
18         4.70      PT  1603672971             
24         4.70      RE  1603672972             
176        0.94      IN  1603672998             
335        4.70      RE  1603673023             
347        4.70      EG  1603673025             
453        0.54      CN  1603673041             
460        1.99      IN  1603673041             
481        2.24      IN  1603673045             
494        4.70      RE  1603673047             
514        2.33      NG  1603673050             
557        5.61      ZA  1603673056

In [28]:
locations = hotel_df[["Lat","Lng"]].astype(float)
hotels = "lodging"
rankby = "distance"

params = {
    "location": locations,  
    "type": "lodging",
    "rankby": "distance",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    data = requests.get(base_url, params=params).json()

    
    try: 
        hotel_name = data["results"][0]["name"]
        print(f"Pulling results for {index}: {hotel_name}")
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except: 
        print("Cant find data for hotel", row["City"])


Pulling results for 18: Estalagem Da Ponta Do Sol


C:\Users\MSI\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Pulling results for 24: Le Palmier
Pulling results for 176: P.J.S.S.V.Mandir Boys Hostel Jagannathpur
Pulling results for 335: Les enfants
Pulling results for 347: مكتبة الطارق للجملة والقطاعي
Pulling results for 453: 红五星宾馆
Pulling results for 460: Circuit House
Pulling results for 481: Hotel Bismillah
Pulling results for 494: The Bird House
Cant find data for hotel Bara
Pulling results for 557: Lekker Bly B&B


In [17]:
hotel_df


City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
18     Ponta do Sol  32.67  -17.10     68.00        64          20   
24   Saint-Philippe -21.36   55.77     71.60        68           0   
176    Jagannāthpur  22.23   85.65     68.22        60           0   
335    Saint-Pierre -21.34   55.48     69.80        68           0   
347            Esna  25.29   32.55     69.80        52           0   
453           Simao  22.79  100.97     69.03        66          20   
460            Tezu  27.92   96.17     74.32        66           0   
481         Sheopur  25.67   76.70     68.32        40           0   
494    Saint-Joseph -21.37   55.62     69.80        68           0   
514            Bara  10.37   10.73     73.78        36          30   
557      Thabazimbi -24.59   27.41     70.00        66           0   

     Wind Speed Country        Date                                 Hotel Name  
18         4.70      PT  1603672971                  Estalagem Da Ponta Do Sol  
24         4.70      RE  1603672972                                 Le Palmier  
176        0.94      IN  1603672998  P.J.S.S.V.Mandir Boys Hostel Jagannathpur  
335        4.70      RE  1603673023                                Les enfants  
347        4.70      EG  1603673025               مكتبة الطارق للجملة والقطاعي  
453        0.54      CN  1603673041                                      红五星宾馆  
460        1.99      IN  1603673041                              Circuit House  
481        2.24      IN  1603673045                            Hotel Bismillah  
494        4.70      RE  1603673047                             The Bird House  
514        2.33      NG  1603673050                                             
557        5.61      ZA  1603673056                             Lekker Bly B&B

In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig2 = gmaps.figure()
fig2.add_layer(gmaps.heatmap_layer(locations, weights = hotel_df["Humidity"]))
fig2.add_layer(markers)
fig2
# Display figure


Figure(layout=FigureLayout(height='420px'))